# toggle-component-path Mapping

In [51]:
import pandas as pd
import numpy as np
toggle_path_df = pd.read_csv('/Users/govardhanrathamsetty/Downloads/1000_comp_matching.csv')
toggle_path_df['tog_id'] = range(1, len(toggle_path_df)+1)
toggle_path_df

,toggle_name,file_path,file_name,version,index,tog_id
0,kUserAgent,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 85.0.4148.0 ...,3535,1
1,kEnableLCDText,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 85.0.4148.0 ...,3942,2
2,kEnableTracing,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 85.0.4148.0 ...,3968,3
3,kSkiaFontCacheLimitMb,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 85.0.4148.0 ...,4045,4
4,kUserAgent,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 85.0.4148.0 ...,4131,5
...,...,...,...,...,...,...
995,kDisableVolumeAdjustSound,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 60.0.3072.0 ...,625431,996
996,kTrySupportedChannelLayouts,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 60.0.3072.0 ...,626318,997
997,kEnableFileCookies,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 60.0.3072.0 ...,626622,998
998,kEnablePinch,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 60.0.3072.0 ...,626642,999


In [52]:
components_df = pd.read_csv('/Users/govardhanrathamsetty/Downloads/components.csv')
components_df

,id,name,source
0,215,chromeos,NaN
1,81,v8,NaN
2,178,native_client,NaN
3,140,notification,NaN
4,98,android,NaN
...,...,...,...
86,193,gpu,NaN
87,199,docs,NaN
88,236,profile,NaN
89,238,reporting,NaN


## start-mapping

In [53]:
path_items=[]
tog_map=[]
path_tog_match =[]

for path in toggle_path_df.file_path:
        
        parts = path.split('/')

        cmpts = parts[8:]
        
        path_items.append(cmpts)
        
path_slicing=[]

for columns, content in toggle_path_df.iteritems():
        if columns == 'file_path':
            parts = content.str.split('/')
            toggle_path_df['cmpts'] = parts.apply(lambda path_slicing: path_slicing[8:11])
toggle_path_df['component_matched'] = None


for i, row1 in toggle_path_df.iterrows():
    for _, row2 in components_df.iterrows():
            for value in row1['cmpts']:
                if value in row2['name']:           
                    toggle_path_df.at[i, 'component_matched'] = row2['name']
                break

toggle_path_df[toggle_path_df.component_matched.notna()]

,toggle_name,file_path,file_name,version,index,tog_id,cmpts,component_matched
0,kUserAgent,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 85.0.4148.0 ...,3535,1,"[ios, chrome, browser]",ios
37,kArcScale,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 85.0.4148.0 ...,25691,38,"[ppapi, shared_impl, ppapi_switches.cc]",ppapi
38,kGpuStartupDialog,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 85.0.4148.0 ...,25954,39,"[ppapi, shared_impl, ppapi_switches.cc]",ppapi
45,kWebXrForceRuntime,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 85.0.4148.0 ...,29869,46,"[chrome, common, chrome_switches.cc]",chromeos
46,kGoogleBaseURL,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 85.0.4148.0 ...,30009,47,"[chrome, common, chrome_switches.cc]",chromeos
...,...,...,...,...,...,...,...,...
783,kDiskCacheSize,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 60.0.3072.0 ...,492917,784,"[ios, chrome, browser]",ios
784,kRecordMode,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 60.0.3072.0 ...,493098,785,"[ios, chrome, browser]",ios
785,kDebugPrint,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 60.0.3072.0 ...,493189,786,"[ios, chrome, browser]",ios
786,kV,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,Switch,chromium 60.0.3072.0 ...,493267,787,"[ios, chrome, browser]",ios


## spread-toggle-extraction

In [55]:
#Remove those rows with Component == None
def spread_toggle(toggle_path_df):
    toggle_path_df = toggle_path_df[toggle_path_df['component_matched'].notna()]
    new_comp=comp.rename(columns={'name': 'component_matched'})
    new_comp = new_comp.drop('source', axis=1)
    spread = pd.concat([group for _, group in toggle_path_df.groupby('toggle_name')])
    spread = spread.merge(new_comp, on='component_matched')
    spread_df = spread.iloc[:, [0, 3, 7, 8]].groupby(['toggle_name', 'component_matched', 'version']).mean().head(20)
    print('Spread Toggle Table:')
    print('--------------------')
    print('--------------------')
    return spread_df
spread_toggle(toggle_path_df)

#return 'spread' to see all spreaded variables.

Spread Toggle Table:
--------------------
--------------------


id
toggle_name                component_matched version                                                  
kAllowExternalPages        ios               chromium 60.0.3072.0                           ...  216.0
kAlsaOutputDevice          chromeos          chromium 80.0.3946.0                           ...  215.0
kApp                       chromeos          chromium 80.0.3946.0                           ...  215.0
                           google_apis       chromium 60.0.3072.0                           ...  105.0
                           ipc               chromium 35.0.1849.0                           ...  135.0
                           ui                chromium 35.0.1849.0                           ...    3.0
kAppId                     google_apis       chromium 60.0.3072.0                           ...  105.0
                           ios               chromium 60.0.3072.0                           ...  216.0
                                             chromium 80.0.3946.0                           ...  216.0
                           ppapi             chromium 35.0.1849.0                           ...  116.0
                                             chromium 80.0.3946.0                           ...  116.0
                           ui                chromium 35.0.1849.0                           ...    3.0
                                             chromium 60.0.3072.0                           ...    3.0
kArcScale                  google_apis       chromium 60.0.3072.0                           ...  105.0
                           ppapi             chromium 35.0.1849.0                           ...  116.0
                                             chromium 80.0.3946.0                           ...  116.0
                                             chromium 85.0.4148.0                           ...  116.0
                           ui                chromium 35.0.1849.0                           ...    3.0
                                             chromium 60.0.3072.0                           ...    3.0
kAshSecondaryDisplayLayout ipc               chromium 40.0.2172.0                           ...  135.0